In [1]:
import numpy as np
import pandas as pd
import catboost

import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
train = pd.read_csv('00/train.csv')
test = pd.read_csv('00/test.csv')
train

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,y
0,b4d8a653ea,16a14a2d17,06330986ed,ca63304de0,a62168d626,1746600cb0,1,1,-0.688706,7e5c97705a,e5df3eff9b,91bb549494,e33c63cf35,3694.0,6e40247e69,617a4ad3f9,718c61545b,c26d08129a,634e3cf3ac,dd9c9e0da2,17c99905b6,513a3e3f36,9aba4d7f51,40.579612,-0.112693,-0.172191,1.166667,1.674538,0.630889,37.000000,1.294922,55.0,0.166667,10.0,0.0,0.000000,1.0,9.0,0.0,1.0,23.0,3.67,0.12,1.935,2.2,0.625,0.250,0.125,0.000,0.813,0.074,0.634,0.548,0.235333,0.264952,0.000000,0.333333,0.333333,0.333333,0.000000,0.000000,9.0,2
1,467f9617a3,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.870871,5624b8f759,fa0b797a92,669ea3d319,f178803074,18156.0,01ede04b4b,617a4ad3f9,718c61545b,d342e2765f,bb20e1ca06,8a6c8cef83,1b02793146,992153ed65,9aba4d7f51,28.765503,2.612285,2.159091,4.000000,1.710714,1.713538,0.166667,0.027669,109.0,0.000000,31.0,0.0,0.000000,1.0,244.0,1.0,1.0,68.0,17.25,0.57,3.452,4.0,0.409,0.619,0.579,0.248,0.346,0.541,0.522,0.000,1.782346,1.322409,0.011647,0.397671,0.239601,0.249584,0.068220,0.033278,601.0,4
2,190436e528,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.437655,5624b8f759,152af2cb2f,91bb549494,e33c63cf35,1178.0,cc69cbe29a,617a4ad3f9,e8a040423a,c82c3dbd33,ee3501282b,199ce7c484,5f17dedd5c,5c5025bd0a,9aba4d7f51,24.943933,-0.814660,-0.708308,1.500000,-0.512422,-0.733967,0.333333,14.837728,11.0,0.000000,24.0,0.0,0.000000,1.0,29.0,0.0,3.0,11.0,4.42,0.15,0.161,0.2,1.000,1.000,1.000,1.000,1.000,0.520,0.533,0.835,-0.586540,0.672436,0.000000,0.606061,0.121212,0.212121,0.060606,0.000000,33.0,3
3,43859085bc,16a14a2d17,06330986ed,ca63304de0,a62168d626,1746600cb0,1,1,0.004439,f67f142e40,c4dd2197c3,91bb549494,e33c63cf35,14559.0,6e40247e69,617a4ad3f9,718c61545b,c26d08129a,9e166b965d,466f8951b0,fde72a6d5c,acfadc5c01,9aba4d7f51,41.576860,-0.907833,-0.761736,0.500000,-0.627525,-0.805801,1.166667,0.004395,0.0,0.500000,0.0,0.0,0.000000,7.0,7.0,0.0,3.0,15.0,8.92,0.29,0.226,0.8,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,-1.600326,-1.838680,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.0,4
4,a4c3095b75,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.480977,7e5c97705a,e071d01df5,91bb549494,e33c63cf35,5777.0,6e40247e69,617a4ad3f9,4b9480aa42,e84655292c,527b6ca8cc,dd9c9e0da2,17c99905b6,0fc56ea1f0,9aba4d7f51,31.080282,-0.371787,-0.367616,1.666667,0.271307,0.013112,17.333333,1713.439128,33.0,0.000000,6.0,1.0,0.666667,8.0,108.0,1.0,4.0,86.0,1.58,0.05,2.032,2.4,0.348,0.762,0.550,0.392,0.489,0.517,1.000,0.642,0.960991,0.790990,0.020161,0.645161,0.258065,0.036290,0.040323,0.000000,248.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,aa2f3cd34a,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,-2.984750,f67f142e40,44dc6ca78b,669ea3d319,b38690945d,1373.0,6e40247e69,617a4ad3f9,718c61545b,759d42196b,969b7983fb,466f8951b0,fde72a6d5c,64ebc0a062,9aba4d7f51,47.320132,-0.348850,-0.363793,5.000000,-0.581484,-0.574904,7.000000,0.003092,0.0,0.000000,2.0,0.0,0.000000,4.0,60.0,0.0,1.0,21.0,8.17,0.27,0.355,0.4,0.222,0.714,0.500,0.000,0.590,0.000,0.000,0.102,1.951098,1.897730,0.264463,0.008264,0.082645,0.438017,0.206612,0.000000,121.0,4
49996,467f9617a3,16a14a2d17,06330986ed,ca63304de0,b7584c2d52,1746600cb0,1,1,0.697585,5624b8f759,eb7575d1e5,91bb549494,e33c63cf35,11496.0,ff390c211c,617a4ad3f9,e8a040423a,0f74e5abc4,f9f4748434,dd9c9e0da2,17c99905b6,11ee6d2a9d,9aba4d7f51,24.179620,-0.244930,-0.217676,3.500000,-0.300509,-0.184992,12.166667,453.386719,32.0,0.666667,18.0,1.0,1.000000,3.0,21.0,0.0,5.0,29.0,2.83,0.09,0.935,1.6,0.800,0.286,1.000,0.572,0.008,1.000,0.829,1.000,0.22

In [3]:
cat_cols = ['x' + str(i) for i in range(10)] + ['x11', 'x12', 'x14', 'x15', 'x16', 'x17', 'x19', 'x20', 'x22', 'x34']
num_cols = ['x' + str(i) for i in range(23, 62) if i not in [34]] 
y = train['y']
y_test = pd.read_csv('00/ans.csv')['y']

X = train.loc[:, cat_cols + num_cols]
test.drop('ID', axis=1, inplace=True)
X_test = test.loc[:, cat_cols + num_cols]


In [4]:
cat_cols_idx = [i for i in range(len(cat_cols))]
num_cols_idx = [i for i in range(len(cat_cols), len(X.columns))]

for col in cat_cols:
    X.loc[:, col] = X.loc[:, col].astype(str)
    X_test.loc[:, col] = X_test.loc[:, col].astype(str)

In [5]:
from sklearn.model_selection import KFold

n_splits = 5
kf = KFold(n_splits=n_splits, random_state=42, shuffle=True)

In [6]:
import catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score

In [7]:
if not os.path.exists('models'):
    os.mkdir('models')

probas = np.zeros((len(X_test), 7))

for i, idx in enumerate(kf.split(X)):
    train_idx, val_idx = idx
    X_train, y_train = X.iloc[train_idx], y[train_idx]
    X_val, y_val = X.iloc[val_idx], y[val_idx]
    
    model = CatBoostClassifier(
            custom_loss=['Accuracy'],
            random_seed=42,
            logging_level='Silent'
        )
    
    if os.path.exists(f'models/model_{i}'):
        model.load_model(f'models/model_{i}')
    else:
        model.fit(
            X_train, y_train,
            cat_features=cat_cols_idx,
            early_stopping_rounds=10,
            eval_set=(X_val, y_val),
            plot=False
        )
    
        model.save_model(f'models/model_{i}')

    probas += np.squeeze(model.predict_proba(X_test))

In [8]:
# simple ensemble

final_preds = np.argmax(probas, axis=1)
accuracy = accuracy_score(y_test, final_preds)
print('The final accuracy is: ', accuracy)

The final accuracy is:  0.76024


In [ ]:
Ссылка на задачу: 
https://special.habrahabr.ru/beeline/